# Format Registered SmartSPIM Datasets for Cloud Viewing

This notebook formulates commands to generate multiscales, chunk, and upload SmartSPIM data to the appropriate AWS S3 bucket.

We can view registered SmartSPIM data and associated label datasets with two cloud viewers:
- Neuroglancer: Multiplanar Reconstruction views presenting orthogonal slices through 3D images
- ITK-VTK-Viewer: Volume rendering of 3D images

Both datasets support spatial multiscales with OME-Zarr. Neuroglancer also supports interactive label viewing with its bespoke "precomputed" format.

See `postprocess_cli.py` for batch image processing and ITK-VTK-Viewer links.

In [ ]:
import os
import re
import sys
from pathlib import Path

import itk
import ngff_zarr

# https://github.com/AllenInstitute/neuroglancer_formatting_scripts
PATH_TO_NEUROGLANCER_INTERFACE = (
    r"C:\repos\neuroglancer_formatting_scripts\src"
)
sys.path.append(PATH_TO_NEUROGLANCER_INTERFACE)
import neuroglancer_interface
from neuroglancer_interface.modules.ccf_multiscale_annotations import (
    write_out_ccf,
)

In [ ]:
IMAGE_FILEPATH = rf"D:\repos\allen-registration\notebooks\data\results\652506\LEVEL_3\2023.06.05\Ex_445_Em_469\labels\652506_Ex_445_Em_469_labels.nii.gz"

IS_LABEL_IMAGE = True
LABELS_DICT_FILEPATH = r"D:\repos\allen-registration\notebooks\data\CCFv31\annotation\itksnap_label_description.txt"

print(
    f'{os.path.basename(IMAGE_FILEPATH)} ({"label" if IS_LABEL_IMAGE else "intensity"})'
)

In [ ]:
LABEL_MESH_FILEPATH = rf'{IMAGE_FILEPATH.rstrip(".nii.gz")}.obj'
LABEL_MESH_FILEPATH

In [ ]:
S3_ROOT_BUCKET_NAME = "aind-kitware-collab"

S3_SAMPLE_BUCKET_NAME = (
    "SmartSPIM_652506_2023-01-09_10-18-12_stitched_2023-01-13_19-00-54"
)
SUBJECT_ID = int(IMAGE_FILEPATH.split("\\results\\")[1].split("\\")[0])
assert str(SUBJECT_ID) in S3_SAMPLE_BUCKET_NAME

REGISTRATION_DATE = "2023.06.05"
REGISTRATION_ID = "L3"

CHANNEL_ID = re.match(".*(Ex_[0-9]*_Em_[0-9]*).*", IMAGE_FILEPATH).group(1)

S3_BUCKET_RESULTS_PATH = f"s3://{S3_ROOT_BUCKET_NAME}/{S3_SAMPLE_BUCKET_NAME}/registered/{REGISTRATION_DATE}/{REGISTRATION_ID}/{CHANNEL_ID}"

if IS_LABEL_IMAGE:
    S3_BUCKET_RESULTS_PATH += "/labels"

if not IS_LABEL_IMAGE:
    TRANSFORM_FILEPATHS = [
        f"{os.path.dirname(IMAGE_FILEPATH)}/{filename}"
        for filename in os.listdir(os.path.dirname(IMAGE_FILEPATH))
        if ".h5" in filename
    ]

print(f"Results will be written to {S3_BUCKET_RESULTS_PATH}")

## Create Surface Mesh

In [ ]:
if IS_LABEL_IMAGE:
    label_image = itk.imread(IMAGE_FILEPATH)
    binary_label_image = itk.binary_threshold_image_filter(
        label_image, lower_threshold=1, inside_value=1
    )
    label_mesh = itk.binary_mask3_d_mesh_source(
        binary_label_image, object_value=1
    )
    itk.meshwrite(label_mesh, LABEL_MESH_FILEPATH, compression=True)

In [ ]:
print(label_mesh)

In [ ]:
def convert_mesh_to_extension(input_filepath: str, extension: str = ".vtp"):
    import vtk

    assert input_filepath.endswith(".obj")

    reader = vtk.vtkOBJReader()
    reader.SetFileName(input_filepath)
    reader.Update()

    print(reader.GetOutput())

    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetInputData(reader.GetOutput())
    writer.SetFileName(f"{os.path.splitext(input_filepath)[0]}{extension}")
    writer.Update()


convert_mesh_to_extension(LABEL_MESH_FILEPATH)

## Convert to OME-Zarr

In [ ]:
OME_ZARR_OUTPUT_FILEPATH = f'{IMAGE_FILEPATH.rstrip(".nii.gz")}.zarr'
print(f"Results will be written to {OME_ZARR_OUTPUT_FILEPATH}")

assert not os.path.exists(OME_ZARR_OUTPUT_FILEPATH)

In [ ]:
# Paste into a shell to generate OME-Zarr downsampling

units_str = " ".join(
    f'"{axis_name}" "millimeter"' for axis_name in ["x", "y", "z"]
)
ngff_zarr_cli_args = (
    f'-i "{IMAGE_FILEPATH}" -o "{OME_ZARR_OUTPUT_FILEPATH}" -u {units_str}'
)

print(f"ngff-zarr {ngff_zarr_cli_args}")

## Convert Labels to Neuroglancer

In [ ]:
if IS_LABEL_IMAGE:
    input_paths = [Path(IMAGE_FILEPATH)]
    NEUROGLANCER_OUTPUT_DIR = Path(
        f"{os.path.dirname(IMAGE_FILEPATH)}/precomputed"
    )
    print(f"Results will be written to {NEUROGLANCER_OUTPUT_DIR}")

In [ ]:
if IS_LABEL_IMAGE:
    os.makedirs(NEUROGLANCER_OUTPUT_DIR, exist_ok=True)

    write_out_ccf(
        segmentation_path_list=input_paths,
        output_dir=NEUROGLANCER_OUTPUT_DIR,
        label_path=LABELS_DICT_FILEPATH,
        use_compression=True,
    )

## Compose S3 Uploads

In [ ]:
if IS_LABEL_IMAGE:
    print(
        f'aws s3 cp --recursive "{os.path.dirname(IMAGE_FILEPATH)}" "{S3_BUCKET_RESULTS_PATH}"'
    )
else:
    for transform_path in TRANSFORM_FILEPATHS:
        print(
            f'aws s3 cp "{transform_path}" "{S3_BUCKET_RESULTS_PATH}/{os.path.basename(transform_path)}"\n'
        )
    print(
        f'aws s3 cp "{IMAGE_FILEPATH}" "{S3_BUCKET_RESULTS_PATH}/{os.path.basename(IMAGE_FILEPATH)}"\n'
    )
    print(
        f'aws s3 cp --recursive "{OME_ZARR_OUTPUT_FILEPATH}" "{S3_BUCKET_RESULTS_PATH}/{os.path.basename(OME_ZARR_OUTPUT_FILEPATH)}"\n'
    )